In [1]:
from bs4 import BeautifulSoup
import requests
import csv
from time import sleep
from random import randint
from datetime import datetime

def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

def get_record(card):
    atag = card.h2.a
    job_title = atag.span.get("title")
    company = card.find("span", {"class": "companyName"}).text
    job_location = card.find("div", {"class": "companyLocation"}).text

    post_date = card.find("span", "date").text
    today = datetime.today().strftime("%Y-%m-%d")
    job_summary = card.find("div", "job-snippet").text
    job_url = 'https://www.indeed.com' + card.h2.a.get("href")
    
    """Below part is added to get the full job description"""
    response1 = requests.get(job_url)    
    soup1 = BeautifulSoup(response1.text, 'html.parser')

    jd = soup1.find('div', 'jobsearch-JobComponent-description icl-u-xs-mt--md')
    
    try:
        full_jd = jd.find("div", "jobsearch-jobDescriptionText").text
    except AttributeError:
        full_jd = ''
        

    
    record = (job_title, company, job_location, post_date, today, job_summary, job_url, full_jd)
    return record

def main(position, location):
    records = []
    url = get_url(position, location)
    
    #while True:       #uncomment this if you want to scrape all the results, it could be slow and take time
    for x in range(1): #Just to scrape one page
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
        
        
    
        for card in cards:
            record = get_record(card)
            records.append(record)
    
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
    
    with open('full_results_1.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title', 'Company', 'Location', 'Posting Date', 'Extract Date', 'Summary', 'Job Url', 'Job Description'])
        writer.writerows(records)

In [2]:
main("data", "ca")

https://www.indeed.com/jobs?q=data&l=ca
